# Merge 2021-12-03

This notebook merges the two datasets from 2022-12-03 into one H5 file. This is for convenient use in the analysis pipeline.

In [ ]:
import sys
import os
from os.path import join
import shutil
from datetime import datetime
import importlib
import json
import numpy as np
import pandas as pd
import h5py
import skimage
from tqdm import trange
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import patches
import proplot as pplt

sys.path.append('../..')
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt

In [ ]:
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['grid'] = False

In [ ]:
folder = '/Diagnostics/Data/Measurements/scan-xxpy-image-ypdE/2021-12-03/'
os.listdir(folder)

In [ ]:
t1 = '211203151919'
t2 = '211203192701'
timestamps = [t1, t2]

Merge the H5 files.

In [ ]:
files = []
for timestamp in timestamps:
    filename = join(folder, timestamp + '-scan-xxpy-image-ypdE.h5')
    files.append(h5py.File(filename))

In [ ]:
nrows = sum([len(file['scandata']) for file in files])

In [ ]:
dtype = files[0]['scandata'].dtype

In [ ]:
file_out = h5py.File(join(folder, f'merged2-{t1}-{t2}-scan-xxpy-image-ypdE.h5'), mode='w')
data_out = file_out.create_dataset('scandata', (nrows,), dtype=dtype)
istart = 0
for file in files:
    data = file['scandata']
    for i in trange(len(data)):
        for name in data.dtype.names:
            data_out[i + istart, name] = data[i, name]
    istart += len(data)
    file.close()
file_out.close()

Copy the csv file for convenience. The csv file in the original data set contains all the planned points.

In [ ]:
shutil.copyfile(
    join(folder, f'{t1}-scan-xxpy-image-ypdE.csv'),
    join(folder, f'merged2-{t1}-{t2}-scan-xxpy-image-ypdE.csv'),
)

Copy the JSON file for convenience.

In [ ]:
shutil.copyfile(
    join(folder, f'{t1}-scan-xxpy-image-ypdE-metadata.json'),
    join(folder, f'merged2-{t1}-{t2}-scan-xxpy-image-ypdE-metadata.json'),
)